<a href="https://colab.research.google.com/github/marbortoli/GoogleColab/blob/main/ImageAI_Object_Detection_by_Moses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get dataset (google glasses from Moses) 
## name: hololens
## image for trial: holo.JPG

!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/hololens.zip

!unzip hololens.zip

In [ ]:
# download a pre trained model - YOLOv3
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

In [ ]:
# necessary support
!pip3 install tensorflow>=2.4.0
!pip install keras>=2.4.3 numpy>=1.19.3 pillow>=7.0.0 scipy>=1.4.1 h5py>=2.10.0 matplotlib>=3.3.2 opencv-python keras-resnet>=0.2.0

# image AI
!pip install imageai --upgrade

In [ ]:
# training model
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer() # class
trainer.setModelTypeAsYOLOv3() # set out model type to YOLOv3
trainer.setDataDirectory(data_directory="hololens") # set the path to my own custom dataset
# specify the parameters: 
### object_names_array: array of the names of all the objects in my dataset
#### ps.: for multiple objects in my annotation folder follow:
##### object_names_array = ["object_name_1", "object_name_2", "object_name_3"]
### batch_size: batch size for the training (larger the batch size - better detection accuracy)
### num_experiments: n of times the train code iterate on my custom dataset
### train_from_pretrained_model: leverage transfer learning using the pretrained YOLOv3 model
trainer.setTrainConfig(object_names_array=["hololens"], batch_size=4, num_experiments=100, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

In [ ]:
# In the sample log shown above, new models are saved based on the decrease in the validation loss. In most cases, the lower the loss, the more accurate the model will be detecting objects in images and videos. However, some models may experience overfitting and still have lower losses. To ensure that you pick the best model for your custom detection, ImageAI allows you to evaluate the mAP (mean Average Precision, read more about it here) of all the trained models saved in the hololens/models folder.
## The higher the mAP, the better the detection accuracy of the model.
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
## other pre trained model types
#trainer.setModelTypeAsRetinaNet()
#trainer.setModelTypeAsTinyYOLOv3()
trainer.setDataDirectory(data_directory='hololens')
trainer.evaluateModel(model_path='hololens/models', json_path='hololens/json/detection_config.json', iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)


**Run custom detection code**

In [ ]:
# get the image to find objects
from google.colab import files
import cv2
import matplotlib.pyplot as plt

uploaded = files.upload()

img = cv2.imread('holo.JPG')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img) 

In [ ]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath('hololens/models/detection_model-ex-034--loss-0017.824.h5') # write the best model's name here
detector.setJsonPath('hololens/json/detection_config.json')
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image='holo.JPG', output_image_path='obj_detected.jpg')

for detection in detections:
  print(detection['name'], " : ", detection['percentage_probability'], " : ", detection['box_points'])
  print('-----')